<a href="https://colab.research.google.com/github/Mahesh29501/Download-Manager/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np

In [15]:
#The Given parameters

total_cust=1000 # 1000 Customers

#Assumptions
#Assuming 90% customers paid on time
cust_no_delay=95/100
# Assuming 2% paid 30 days after due date
cust_30_days=2/100 
# Assuming 2% paid 45 days after due date
cust_45_days=2/100 
# Assuming 1% paid 60 days after due date
cust_60_days=1/100

#Monthly average balance
month_bal=1000

# Source of Revenue per year
mem_fee=12*20*total_cust # membership fee is Rs 20 per card
affil_fee=12*10*total_cust # Rs 10 affiliation fee per card
int_rate=15/100 # Interest rate

# Expenses per year
cost_of_funds=12*total_cust*1000*0.065/12 # Bank lending money to the credit card company
operating_cost=12*25*1000 # Rs 25 operating cost per card
loss_rate=3/100 # 3% loss due to non-payment of credit card bills

In [16]:
# Category Table based on default date

day=30
categories=4
category_type=[]

for cust_category in range(categories):
    if cust_category==0:
        category_type.append('Paid before due date')
    else:
        category_type.append('Paid %d days after due date'%day)
        day+=15     

Data_Calculations=pd.DataFrame({'Category':category_type},index=np.arange(1,categories+1))
Data_Calculations

,Category
1,Paid before due date
2,Paid 30 days after due date
3,Paid 45 days after due date
4,Paid 60 days after due date


In [17]:
#Interest after 'N' days = N*outstanding balance*Interest rate per year/365

def interest_calc(payment_days):
    
    # Billing cycle starts from 1st of every month
    average_day_trans= 15 #Assuming average day of transaction as 15th of billing cycle 
    
    if payment_days==category_type[0]:
        Int_per_card=0

    #Interest for 30 days
    elif payment_days==category_type[1]:
        Int_per_card=(average_day_trans+15+30)*month_bal*int_rate/365

    #Interest for 45 days
    elif payment_days==category_type[2]:
        Int_per_card=(average_day_trans+15+45)*month_bal*int_rate/365

    #Interest for 60 days
    else:
        Int_per_card=(average_day_trans+15+60)*month_bal*int_rate/365
        
    return Int_per_card 
    
def total_int(column):
    
    payment_days=column[0]
    interest_per_card=column[1]
    
    if payment_days==category_type[0]:
        total_interest=0
    #Interest for all customers: late by 30 days
    elif payment_days==category_type[1]:
        total_interest=12*interest_per_card*cust_30_days*total_cust
    #Interest for all customers: late by 45 days
    elif payment_days==category_type[2]:
        total_interest=12*interest_per_card*cust_45_days*total_cust
   #Interest for all customers: late by 60 days, remove these customers after non payment
    else:
        total_interest=12*interest_per_card*cust_60_days*total_cust
        
    return total_interest

def PnL_check(): # Column name based on profit/loss
    if net_profit_loss>=0:
        return 'Profit margin in percentage'
    else:
        return 'Loss margin in percentage'

Data_Calculations['Estimated_Interest_on_Default']=Data_Calculations['Category'].apply(interest_calc).round(2)
Data_Calculations['Total_Interest_1Year']=Data_Calculations[['Category','Estimated_Interest_on_Default']].apply(total_int,axis=1)
Data_Calculations['Fixed_Late_Fee_charge']=Data_Calculations['Estimated_Interest_on_Default'].apply(lambda x: 0 if x==0 else 50)
Data_Calculations

,Category,Estimated_Interest_on_Default,Total_Interest_1Year,Fixed_Late_Fee_charge
1,Paid before due date,0.00,0.0,0
2,Paid 30 days after due date,24.66,5918.4,50
3,Paid 45 days after due date,30.82,7396.8,50
4,Paid 60 days after due date,36.99,4438.8,50


In [18]:
#PROFIT GENERATED

Expenses=cost_of_funds+operating_cost+(loss_rate*total_cust*month_bal)
Earnings=mem_fee+affil_fee+np.sum(Data_Calculations['Total_Interest_1Year'])+(12*max(Data_Calculations['Fixed_Late_Fee_charge'])*5/100*total_cust)

margin=Earnings-Expenses # margin earned above or below total amount spent
net_profit_loss=margin/Expenses*100
    
report=pd.DataFrame({'Total yearly card balance':12*month_bal*total_cust,'Earnings':Earnings,'Expenses':Expenses,
              'Margin':margin,PnL_check():net_profit_loss.round(2)},index=[' '])
report

,Total yearly card balance,Earnings,Expenses,Margin,Profit margin in percentage
,12000000,407754.0,395000.0,12754.0,3.23


# <center>The Profit for the given Case Study is approximately *3.2%</center>*
<hr>
<hr>

**Insights :**


*   Earnings to expense ratio is: 1.032
*   Loss rate (loss incurred due to non payment of bills by customers) increases with increase in average card balance and as a result overall profits are also impacted.
*   The more number of customers that defaults or pay after due date, the more profit a company generates in the form of Late fee/Interest charges. So, the company benefits from credit defaults considering 95-99% customers settle the amount




